Udemy Course Scraper

In [ ]:
# I worry that all of these xpath links will get broken by the react they're using. They had strange names.

# Change this variable if you're running all and want to overwrite existing cookies
force_new_cookies = False

# Parameters
cookies_path = "cookies.json"
purchase_page_pause = 20 # It can take a moment to load (increase or decrease this depending on your internet)
course_page_pause = 5 # Adjust this as well
refund_page_wait = 5 # Adjust this as well
course_grid_wait = 5 # Adjust this as well
cart_pause = 10 # Adjust as well
store_page_pause = 5 # Adjust as well
max_course_size = 50 # Max cart for Udemy, adjust lower if you want
random_wait_min = 1
random_wait_max = 5

# Options for selenium
import undetected_chromedriver as uc
from undetected_chromedriver import By
from typing import Union, Tuple, Any
import pandas as pd
import time
import json
import os
import datetime
from decimal import Decimal
import re
import random

options = uc.ChromeOptions()
driver = uc.Chrome(use_subprocess=False, options=options)

# Go to udemy
driver.get("https://www.udemy.com/")

Manually log into Udemy now because trying to log in automatically with 2FA and everything will be a pain in the ass. Save the cookies to skip this step later.

I tried to export my cookies from Chrome and load them into here before, but it didn't work. So be it.

Cookie Functions

In [ ]:
def saveCookies(t_driver: uc.Chrome, f_path: str, force_cookies: bool = True) -> None:
    '''
    Saves the cookies from your udemy in

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param f_path: The file name (or full file path)
    :type f_path: str

    :param force_cookies: If we force the cookies even if they already exist, defaults to [True]
    :type force_cookies: bool

    :return: Nothing
    :rtype: None
    '''
    # Checking if the cookies file already exists
    if(force_cookies or not os.path.exists(f_path)):
        # Wait for an input for the user to log in
        input("Continue after you've logged into Udemy!")
        # To save cookies
        saved_cookies = t_driver.get_cookies()
        with open(f_path, "w") as file:
            json.dump(saved_cookies, file)
    return None
def loadCookies(t_driver: uc.Chrome, f_path: str):
    '''
    Loads your Udemy cookies in

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param f_path: The file name (or full file path)
    :type f_path: str

    :return: Nothing
    :rtype: None
    '''
    # To load in cookies
    with open(f_path, "r") as file:
        saved_cookies = json.load(file)
    for x in saved_cookies:
        t_driver.add_cookie(x)
    # Wait for one second just to be sure (probably not needed)
    time.sleep(1)
    return None

General Functions

In [ ]:
def checkMultipleCourses(web_element: uc.WebElement) -> Union[bool, uc.WebElement]:
    '''
    Checks if a purchase is a single course or multiple courses

    :param web_element: The selenium web_element
    :type web_element: WebElement

    :return: Either False or the WebElement to the button to show more
    :rtype: bool or WebElement
    '''
    try:
        temp = web_element.find_element(By.XPATH, ".//button[@class='ud-btn ud-btn-medium ud-btn-link ud-heading-sm show-more_focusable-label__dmEhW']")
        return(temp)
    except:
        return(False)
# Function to check if a refund button exists for a course
def checkRefund(web_element: uc.WebElement) -> Union[bool, uc.WebElement]:
    '''
    Checks if a course has a refund (inside of a larger purchase, not individual courses)

    :param web_element: The selenium web_element
    :type web_element: WebElement

    :return: Either False or the WebElement to the refund button
    :rtype: bool or WebElement
    '''
    try:
        # This will fail if the element can't be found
        temp = web_element.find_element(By.XPATH, ".//div[@data-testid='redirect-button']")
        temp = temp.find_element(By.XPATH, ".//a")
        return(temp)
    except:
        return(False)
def getRefundCourses(t_courses_list: list) -> list:
    '''
    The function to get courses that have refunds available

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :return: The list of just courses with refunds available
    :rtype: list
    '''
    # Get courses with refund links
    temp_list = [course for course in t_courses_list if course["Refund_Link"] not in ["Refunded", "No Refund Available"]]
    # for x in t_courses_list:
    #     if(x["Refund_Link"] not in ["Refunded", "No Refund Available"]):
    #         temp_list.append(x)
    return(temp_list)
def getCoursesToPurchase(t_courses_list: list) -> list:
    '''
    The function to get courses to purchase

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :return: The list of just courses with refunds available
    :rtype: list

    '''
    # Get courses with refund links
    temp_list = [course for course in t_courses_list if course["Refund_Status"] in ["Prior", "Already Refunded or Unowned", "Refunded"]]
    # for x in t_courses_list:
    #     if(x["Refund_Link"] not in ["Prior", "Already Refunded or Unowned", "Refunded"]):
    #         temp_list.append(x)
    return(temp_list)
def saveJsonItem(path_to_file: str, object_to_write: Any, overwrite: bool = True) -> None:
    '''
    The function to save JSON objects

    :param path_to_file: The path to the file to save at
    :type path_to_file: str

    :param object_to_write: Any object to write (in json)
    :type object_to_write: any

    :param overwrite: If we force the overwriting of existing files, defaults to [True]
    :type overwrite: bool

    :return: Nothing
    :rtype: None
    '''
    if(overwrite or not os.path.exists(path_to_file)):
        with open(path_to_file, "w") as file:
            # Quick and dirty because apparently JSON can't serialize datetimes or dates and I
            # don't want to spend anymore time writing functions to do that properly (in the default parameter)
            json.dump(object_to_write, file, indent=4, sort_keys=True, default=str)
    return None
def loadJsonItem(path_to_file: str) -> Tuple[dict, list]:
    '''
    The function to load JSON objects

    :param path_to_file: The path to the file to save at
    :type path_to_file: str

    :return: A dictionary or a list depending on the data
    :rtype: dict or list
    '''
    with open(path_to_file, "r") as file:
        loaded_data = json.load(file)
    return loaded_data
def saveListItem(path_to_file: str, object_to_write: list, overwrite: bool = True) -> None:
    '''
    The function to save lists

    :param path_to_file: The path to the file to save at
    :type path_to_file: str

    :param object_to_write: Any object to write (in json)
    :type object_to_write: any

    :param overwrite: If we force the overwriting of existing files, defaults to [True]
    :type overwrite: bool

    :return: Nothing
    :rtype: None
    '''
    if(overwrite or not os.path.exists(path_to_file)):
        with open(path_to_file, "w") as file:
            file.writelines([item + "\n" for item in object_to_write], file)
    return None
def loadListItem(path_to_file: str) -> list:
    '''
    The function to load list objects

    :param path_to_file: The path to the file to save at
    :type path_to_file: str

    :return: A list
    :rtype: list
    '''
    with open(path_to_file, "r") as file:
        loaded_data = file.readlines()
        # Remove the /n
        #loaded_data = [x[:-1] for x in loaded_data]
        # Well I suppose it could be elsewhere (even though it shouldn't be in my data)
        loaded_data = [x.replace("\n","") for x in loaded_data]
    return loaded_data

Function for getting purchase information per page

In [ ]:
def scrapePurchasePage(t_purchases_list: list, t_courses_list: list, t_driver: uc.Chrome) -> Tuple[list, list]:
    '''
    The main function to scrape the individual purchase pages

    :param t_purchases_list: The list to store the purchases
    :type t_purchases_list: list

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :param t_driver: The web driver
    :type t_driver: Chrome

    :return: The lists you input but with the additional purchases and courses appended [t_purchases_list, t_courses_list]
    :rtype: Tuple[list, list]

    '''
    # Get the purchases
    purchase_table = t_driver.find_element(By.XPATH, "//table[@class='paginated-table_purchase-history-table__ncCll']")
    purchases = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--main-row__5hHCb']")

    for purchase in purchases:
        # Get the purchase
        temp_purchase = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text-container__EvFRz']")
        # Get the receipt
        receipt = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Receipt')]")
        receipt = receipt.find_element(By.XPATH, "..")
        receipt = receipt.get_attribute("href")
        receipt = receipt.split("/")[-2]
        # Get the invoice (removing because not every purchase has an invoice and the receipt is what is useful anyway)
        # invoice = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Invoice')]")
        # invoice = invoice.find_element(By.XPATH, "..")
        # invoice = invoice.get_attribute("href")
        # invoice = invoice.split("/")[-2]
        total_price = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L']/div[@class='app_positive-text___Wb07']").text
        # Using decimal even though floats would work for this purpose
        total_price = Decimal(total_price.replace("$","").strip())
        # Get the data ( I couldn't also done find_elements and selected the index 1, but this should be better and was neat to learn)
        purchase_date = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L' and not(descendant::*)]").text
        purchase_date = datetime.datetime.strptime(purchase_date, "%b %d, %Y").date()

        # Get the purchase number (may not be that efficient, but oh well)
        p_ind = len(t_purchases_list) + 1

        # Add the purchase
        cur_purchase = {"Date": purchase_date, 
                        "Purchase": p_ind,
                        "Price": total_price,
                        "Receipt": receipt}
        t_purchases_list.append(cur_purchase)

        # Check if we need to check for a refund or not (over 30 days won't have the ability to return)
        check_refund_bool = False
        current_date = datetime.datetime.now().date()
        if((current_date - purchase_date).days <= 30):
            check_refund_bool = True

        # Check if it has multiple purchases
        temp_multiple = checkMultipleCourses(temp_purchase)
        if temp_multiple != False:
            # Click open the courses
            temp_multiple.click()
            # Wait for it to load because it can take a moment
            time.sleep(1)
            # Get the individual courses
            courses = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--sub-row__2g87b']")
            for course in courses:
                # Check if it has a refund button
                if check_refund_bool:
                    temp_link = checkRefund(course)
                    if(temp_link != False):
                        refund_link = temp_link.get_attribute("href")
                    else:
                        refund_link = "Refunded"
                else:
                    refund_link = "No Refund Available"
                course_name = course.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
                course_name = course_name.find_element(By.XPATH, ".//a")
                c_name = course_name.get_attribute("innerHTML")
                c_link = course_name.get_attribute("href")
                course_price = course.find_element(By.XPATH, ".//div[@class='app_positive-text___Wb07']").text
                course_price = Decimal(course_price.replace("$","").strip())

                # Get the course number (may not be that efficient, but oh well)
                c_ind = len(t_courses_list) + 1
                
                # Add the course
                cur_course = {"Date": purchase_date, 
                                "Purchase": p_ind,
                                "Price": course_price,
                                "Receipt": receipt,
                                "Course": c_name,
                                "Course_Num": c_ind,
                                "Course_Link": c_link,
                                "Refund_Link": refund_link}
                t_courses_list.append(cur_course)
            # Close what we opened because the subrows are inserted directly into it
            temp_multiple.click()
        # If it's a singular course
        else:
            if check_refund_bool:
                temp_link = checkRefund(temp_purchase)
                if(temp_link != False):
                    refund_link = temp_link.get_attribute("href")
                else:
                    refund_link = "Refunded"
            else:
                refund_link = "No Refund Available"
            course_name = temp_purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
            course_name = course_name.find_element(By.XPATH, ".//a")
            c_name = course_name.text
            c_link = course_name.get_attribute("href")

            # Get the course number (may not be that efficient, but oh well)
            c_ind = len(t_courses_list) + 1

            # Add the course
            cur_course = {"Date": purchase_date, 
                            "Purchase": p_ind,
                            "Price": total_price,
                            "Receipt": receipt,
                            "Course": c_name,
                            "Course_Num": c_ind,
                            "Course_Link": c_link,
                            "Refund_Link": refund_link}
            t_courses_list.append(cur_course)
    return(t_purchases_list, t_courses_list)

Function to get the purchases

In [ ]:
def scrapePurchases(t_purchases_list: list, t_courses_list: list, t_driver: uc.Chrome) -> Tuple[list, list]:
    '''
    The function to handle multiple pages of purchases

    :param t_purchases_list: The list to store the purchases
    :type t_purchases_list: list

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :param t_driver: The web driver
    :type t_driver: Chrome

    :return: The lists you input but with the additional purchases and courses appended
    :rtype: Tuple[list, list]

    '''
    # Go to the purchases page initially
    t_driver.get("https://www.udemy.com/dashboard/purchase-history/")

    try:
        # Inside a while loop because the amount of pages is unknown
        while True:
            # Wait for the page to load
            time.sleep(purchase_page_pause + random.randint(random_wait_min, random_wait_max))
            # Scrape the page
            t_purchases_list, t_courses_list = scrapePurchasePage(t_purchases_list, t_courses_list, t_driver)
            # Get the next page button (it won't find it when we've done the last page)
            next_page = t_driver.find_element(By.XPATH, ".//a[@class='ud-btn ud-btn-medium ud-btn-secondary ud-btn-round ud-heading-sm ud-btn-icon ud-btn-icon-medium ud-btn-icon-round pagination_next__aBqfT'" \
            "and @aria-label='next page' and @aria-disabled='false']")
            next_page.click()
            
    # When the next page button isn't found return
    except:
        return(t_purchases_list, t_courses_list)
        
    

Function to open the course and see what the current price is (Logged Out for better sales)

In [ ]:
def checkCoursePrice(t_driver: uc.Chrome, t_courses_list: list, new_prices: bool = True) -> list:
    '''
    The function to get the current course prices

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param t_courses_list: The list of courses with refunds available
    :type t_courses_list: list

    :param new_prices: If True will delete all cookies to get the new user sales 
    (which are usually cheaper than an existing user. You can still
    purchase courses from being logged out which keeps that coupon)
    :type new_prices: bool

    :return: The list of just courses with refunds available
    :rtype: list

    '''
    # Delete all cookies to get the new user sale price
    if(new_prices):
        t_driver.delete_all_cookies()
    # Go the course
    for ind, course in enumerate(t_courses_list):
        t_driver.get(course["Course_Link"])
        time.sleep(course_page_pause)
        cur_price = t_driver.find_element(By.XPATH, ".//div[@data-purpose='course-price-text']")
        # cur_price = t_driver.find_element(By.XPATH, "//span[@class='ud-sr-only' and contains(., 'Current price')]")
        cur_price = cur_price.find_element(By.XPATH, ".//span[not(@class='ud-sr-only')]").get_attribute("innerHTML")
        cur_price = Decimal(re.sub("(<span>)|(<\/span>)", "", cur_price).replace("$","").strip())
        course["Sale_Price"] = cur_price
        t_courses_list[ind] = course
    # Get courses that are cheaper
    t_courses_list = [course for course in t_courses_list if course["Sale_Price"] < course["Price"]]
    return(t_courses_list)

Function to refund the course

In [ ]:
def refundCourse(t_driver: uc.Chrome, t_courses_list: list) -> list:
    '''
    The function to refund courses

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param t_courses_list: The list of courses with refunds available that are cheaper
    :type t_courses_list: list

    :param new_prices: If True will delete all cookies to get the new user sales 
    (which are usually cheaper than an existing user. You can still
    purchase courses from being logged out which keeps that coupon)
    :type new_prices: bool

    :return: The list of just courses with refunds available
    :rtype: list
    '''
    # Forgive how nested this is. I can't be bothered to properly functionalize all of this

    running_refund = Decimal(0)
    running_price = Decimal(0)
    running_savings = Decimal(0)

    avail_credits = Decimal(0)
    avail_price = Decimal(0)
    avail_savings = Decimal(0)
    real_credits = Decimal(0)

    # Go the course
    for ind, course in enumerate(t_courses_list):
        while True:
            t_driver.get(course["Refund_Link"])
            time.sleep(refund_page_wait + random.randint(random_wait_min, random_wait_max))
            # Check to make sure it'll let us refund (wont let you refund multiple times)
            try:
                refunded_prior = t_driver.find_element(By.XPATH, "//span[@data-purpose='safely-set-inner-html:refund-error:message']").text
                # Check if it was refunded prior
                if("Multiple refund requests" in refunded_prior):
                    course["Refund_Status"] = "Prior"
                    t_courses_list[ind] = course
                    print("Course {} Prior Refund at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    break
            except:
                # Just keep going
                pass
            # Check if the course has already been refunded / you don't own it
            try:
                already_refunded = t_driver.find_element(By.XPATH, "//div[@class='ud-text-md app--intro-text---dQW-']")
                already_refunded = already_refunded.find_element(By.XPATH, ".//span").text
                if("how to refund a course" in already_refunded):
                    course["Refund_Status"] = "Already Refunded or Unowned"
                    t_courses_list[ind] = course
                    print("Course {} Unowned or Already Refunded at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    break
            except:
                # Just keep going
                pass
            original_avail_savings = avail_savings
            failed_refund = False
            try:
                avail_savings = avail_savings + course["Price"] - course["Sale_Price"]
                # I want to return to Paypal every time I have enough for the current
                # course in credits (with some wiggle room so we don't end up with a surplus of credits)
                temp_log_bool = False
                if(original_avail_savings >= (course["Sale_Price"]-Decimal(0.5))):
                    avail_savings = original_avail_savings - course["Sale_Price"]
                    # If it's less than 0 set it to 0 so there' no negatives (I'd rather have more refunded than extra credits)
                    if(avail_savings < Decimal(0)):
                        avail_savings = Decimal(0)
                    original_refund = t_driver.find_element(By.XPATH, ".//button[@data-purpose='show-original-payment-method']")
                    original_refund.click()
                    # Wait just in case
                    time.sleep(1)
                    temp_log_bool = True
                # Select the reason (a little tricky with the select option. Apparently there's a Select() option
                # in Selenium, but I couldn't find it in Python. It's in selenium.webdriver.support.ui apparently, but
                # doesn't look like undetected_chromedriver have the option. Maybe you can still use it from Selenium though)
                refund_reason = t_driver.find_element(By.XPATH, ".//div[@data-purpose='refund-reason-selection']")
                refund_reason = refund_reason.find_element(By.XPATH, ".//div[@class='ud-select-container ud-select-container-medium']")
                refund_reason = refund_reason.find_element(By.XPATH, ".//select")
                refund_reason = refund_reason.find_elements(By.XPATH, ".//option")[5]
                time.sleep(1)
                refund_reason.click()
                # Selecting the reason
                refund_description = t_driver.find_element(By.XPATH, ".//textarea[@class='ud-text-input ud-text-area text-area-module--ud-text-area-large--2--TA ud-text-md']")
                refund_reasons = [
                    "Course is cheaper now",
                    "Cheaper now",
                    "On sale now",
                    "Cheaper",
                    "The course is cheaper now",
                    "Cheaper course",
                    "Cheaper course now"
                ]
                # Get a random phrase
                cur_reason = random.choice(refund_reasons)
                # Simulate not hitting shift
                if(random.randint(0,1) == 1):
                    cur_reason = cur_reason.lower()
                # Simulate not hitting space
                if(random.randint(0,1) == 1):
                    cur_reason = re.sub(" ", "", cur_reason, random.randint(1,2))
                # Simulate typos
                if(random.randint(0,1) == 1):
                    last_cut = len(cur_reason)
                    for x in range(random.randint(1,3)):
                        ran_cut = random.randint(0,len(cur_reason)-1)
                        if(ran_cut != last_cut):
                            cur_reason = cur_reason[:ran_cut] + cur_reason[ran_cut+1:]
                            last_cut = ran_cut
                # Set the description
                refund_description.send_keys(cur_reason)
                # Selecting the submit button
                refund_button = t_driver.find_element(By.XPATH, ".//button[@data-purpose='submit-refund-form']")
                refund_button.click()
                # Wait for the refund to process
                time.sleep(refund_page_wait + random.randint(random_wait_min, random_wait_max))
                # Apparently the refund problem actually shows up after the refund goes through so I need to check that
                try:
                    refunded_prior = t_driver.find_element(By.XPATH, "//span[@data-purpose='safely-set-inner-html:refund-error:message']").text
                    avail_savings = original_avail_savings
                    # Check if we're on a break
                    if("try again later" in refunded_prior):
                        # Sleep for 2.5 minutes and try again (not sure how long it actually makes you wait, I had 5 originally, but I'll reduce it and see)
                        print("Course {} Refund Break at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                        time.sleep(2.5 * 60)
                except:
                    # Check for a success
                    try:
                        success_text = t_driver.find_element(By.XPATH, "//h1[@class='ud-heading-serif-xxl']").text
                        if(success_text == "Thank you"):
                            ## I might want to try here to check for a success actually
                            # Tracking the total run
                            running_refund = running_refund + course["Price"]
                            running_price = running_price + course["Sale_Price"]
                            running_savings = running_refund - running_price
                            # Tracking for when to refund to PayPal vs Udemy
                            avail_credits = avail_credits + course["Price"]
                            avail_price = avail_price + course["Price"]
                            real_credits = avail_credits - avail_price
                            # Add to the data
                            course["Refund_Status"] = "Refunded"
                            course["Running_Refund"] = running_refund
                            course["Running_Price"] = running_price
                            course["Running_Savings"] = running_savings
                            course["Avail_Credits"] = avail_credits
                            course["Avail_Price"] = avail_price
                            course["Avail_Savings"] = avail_savings
                            course["Real_Credits"] = real_credits
                            t_courses_list[ind] = course
                            temp_log_out = "Credits"
                            if(temp_log_bool):
                                temp_log_out = "Refunded"
                            # Print for testing
                            print(f'Course {ind} : Current Credits {avail_savings} : {temp_log_out} : Successful Refund at: {datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")}')
                            # Break the while loop
                            break
                        else:
                            failed_refund = True
                            raise Exception("Not Successful Refund")
                    except:
                        # Believe that this will break the try this is inside of
                        raise Exception("Go to other Except")
            except:
                # If we failed for some reason reset the avail_savings just in case it was modified
                avail_savings = original_avail_savings
                if(failed_refund):
                    print("Course {} Unsuccessful Refund at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    time.sleep(0.5 * 60)
                else:
                    # Check if we got a refund break (leaving this here just in case this page does appear)
                    try:
                        on_break = t_driver.find_element(By.XPATH, "//span[@data-purpose='safely-set-inner-html:refund-error:message']").text
                        # Check if we're on a break
                        if("try again later" in on_break):
                            # Sleep for 2.5 minutes and try again (not sure how long it actually makes you wait, I had 5 originally, but I'll reduce it and see)
                            print("Course {} Refund Break at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                            time.sleep(2.5 * 60)
                    # If it's not a refund break it's unknown and we break still (might want to crash the script or something instead)
                    except:
                        course["Refund_Status"] = "Error"
                        t_courses_list[ind] = course
                        print("Course {} Unknown Refund Failed at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                        break
    return(t_courses_list)

Function to add a course to your cart

In [ ]:
def addToCart(t_driver: uc.Chrome, t_courses_list: list, new_prices: bool = True) -> list:
    '''
    The function to add cheaper courses to the cart

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param t_courses_list: The list of courses with refunds available that are cheaper
    :type t_courses_list: list

    :param new_prices: If True will delete all cookies to get the new user sales 
    (which are usually cheaper than an existing user. You can still
    purchase courses from being logged out which keeps that coupon)
    :type new_prices: bool

    :return: The list of just courses with refunds available
    :rtype: list

    '''
    # Delete all cookies to get the new user sale price
    if(new_prices):
        t_driver.delete_all_cookies()
    # Track courses because Udemy has a limit
    cart_size = 0
    # Go the course
    for ind, course in enumerate(t_courses_list):
        while True:
            t_driver.get(course["Course_Link"])
            time.sleep(course_page_pause + random.randint(random_wait_min, random_wait_max))
            # Check if there's an add to cart button
            try:
                cart_button = t_driver.find_element(By.XPATH, ".//div[@class='buy-box--buy-box-item--wT5bJ buy-box--add-to-cart-button-wrapper--focmP']")
                cart_button = cart_button.find_element(By.XPATH, ".//button[@data-testid='add-to-cart-button']")
                # For som reason the selenium.click() won't work with this button, but executing it as JavaScript does.
                t_driver.execute_script("arguments[0].click()", cart_button)
                cart_size = cart_size + 1
                time.sleep(cart_pause)
                print("Course {} Added to cart at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                # Check the cart size
                if(cart_size == max_course_size):
                    input("Cart Full! Make a purchase and continue!")
                    # Delete all cookies to get the new user sale price
                    if(new_prices):
                        t_driver.delete_all_cookies()
                    cart_size = 0
                course["New_Status"] = "Added to Cart / Purchased"
                t_courses_list[ind] = course
                break
            except:
                # Check if we already own the course
                try:
                    course_button = t_driver.find_element(By.XPATH, ".//div[@class='buy-button buy-box--buy-box-item--wT5bJ buy-box--buy-button--m373K']")
                    course_button = course_button.find_element(By.XPATH, ".//button[@data-purpose='buy-this-course-button']")
                    course_button = course_button.find_element(By.XPATH, ".//span[@class='ud-btn-label']").text
                    print("Course {} Already Owned at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    course["New_Status"] = "Owned"
                    t_courses_list[ind] = course
                    break
                # If there's some kind of error we'll retry
                except:
                    print("Course {} Error at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    course["New_Status"] = "Error"
                    t_courses_list[ind] = course
                    break
    return(t_courses_list)



Function to scrape the courses you own (and maybe have taken if on the subscription)

In [ ]:
def scrapeCourseInformation(t_driver: uc.Chrome, courses_link: str, course_list: Union[list, None] = None) -> list:
    '''
    The main function to scrape the owned courses

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param courses_link: The link for the Udemy Courses (a parameter so you can do All Courses and Archived)
    :type courses_link: str

    :param course_list: A list to add the courses to. An empty list will work (None or missing will create an empty list)
    :type course_list: list or None

    :return: The lists you input but with the additional purchases and courses appended [t_purchases_list, t_courses_list]
    :rtype: list

    '''
    if course_list == None:
        course_list = []
    # Go to the link
    t_driver.get(courses_link)
    while True:
        time.sleep(course_grid_wait + random.randint(random_wait_min, random_wait_max))
        try:
            courses_grid = t_driver.find_element(By.XPATH, "//div[@class='my-courses__course-card-grid']")
            # Note on this // gets all children, / gets only the next level!
            # I could just iterate over this, but I want to try out the xpath siblings stuff... actually that'd be stupid for this purpose
            courses = courses_grid.find_elements(By.XPATH, "./div")
            #courses = courses_grid.find_element(By.XPATH, "./div")
            # siblings = courses.find_element(By.XPATH, "following-sibling::*")
            for course in courses:
                course_info = course.find_element(By.XPATH,".//h3[@data-purpose='course-title-url']//a")
                instructor = course.find_element(By.XPATH, ".//div[@class='course-card-instructors-module--instructor-list--cJTfw']").text
                instructor_trunc = re.split(r"[,\|•-]", instructor)[0].strip()
                course_list.append({"Course": course_info.text,
                                        "Instructors": instructor,
                                        "Instructor_Trunc": instructor_trunc,
                                        "Course_Link": course_info.get_attribute("href")
                                        }) 
        except:
            print("No courses found at: {}".format(datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
            return course_list
        try:  
            next_page = t_driver.find_element(By.XPATH, ".//a[@class='ud-btn ud-btn-medium ud-btn-secondary ud-btn-round ud-heading-sm ud-btn-icon ud-btn-icon-medium ud-btn-icon-round pagination-module--next--QX6nm'"
            " and @aria-label='next page' and @aria-disabled='false']")
            next_page.click()
        except:
            break
    return course_list

Function to get the link to the store page from the owned (possible subscription) courses

In [ ]:
def scrapeCourseStorePages(t_driver: uc.Chrome, course_list: list) -> list:
    '''
    The main function to scrape the individual purchase pages

    :param t_driver: The web driver
    :type t_driver: Chrome
    
    :param course_list: A list to add the courses to. An empty list will work (None or missing will create an empty list)
    :type course_list: list

    :return: The lists you input with the link to the store page added
    :rtype: list

    '''
    for ind, course in enumerate(course_list):
        try:
            t_driver.get(course["Course_Link"])
            time.sleep(store_page_pause + random.randint(random_wait_min, random_wait_max))
            course_link = t_driver.find_element(By.XPATH, "//a[@class='ud-text-md header--header-text--zBvgT header--header-link--X0YLd truncate-with-tooltip--ellipsis--YJw4N ud-text-md header--header-text--zBvgT header--header-link--X0YLd']")
            course["Store_Link"] = course_link.get_attribute("href")
        except:
            course["Store_Link"] = "Error"
        course_list[ind] = course
    return(course_list)

Handle Cookies

In [ ]:
saveCookies(driver, cookies_path, force_new_cookies)
loadCookies(driver, cookies_path)

Get the purchases and courses

In [ ]:
# Get the purchases and courses
# Items to store the purchases (would probably be better to do SQLite or DuckDB for persistence, but this will work for now)
# purchases_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice"])
# courses_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice", "Course"])
purchases_list = []
courses_list = []

purchases_list, courses_list =  scrapePurchases(purchases_list, courses_list, driver)

# Save in case of crash you can reload them
saveJsonItem("purchases.json", purchases_list)
saveJsonItem("courses.json", courses_list)

Get the courses that have refunds available

In [ ]:
refund_available = getRefundCourses(courses_list)
saveJsonItem("refund_available.json", refund_available)

Check the prices of the courses

In [ ]:
cheaper_refund = checkCoursePrice(driver, refund_available)
saveJsonItem("cheaper_refund.json", cheaper_refund)
loadCookies(driver, cookies_path)

Refund Courses

In [ ]:
refunded_courses = refundCourse(driver, cheaper_refund)
saveJsonItem("refunded_courses.json", refunded_courses)

Get the courses to Purchase

In [ ]:
courses_to_purchase = getCoursesToPurchase(refunded_courses)

Getting the owned course information

In [ ]:
owned_courses = scrapeCourseInformation(driver, "https://www.udemy.com/home/my-courses/learning/")
owned_courses = scrapeCourseInformation(driver, "https://www.udemy.com/home/my-courses/archived/", owned_courses)
# This would actually probably take forever... we can probably just join
# by the course names if both the purchases and courses were scraped
# close to each other. Well I suppose we still would because 2 courses
# could have the same name, but not the same link.
#owned_courses = scrapeCourseStorePages(driver, owned_courses)
saveJsonItem("owned_courses.json", owned_courses)

Remove any already owned courses that somehow got in there

In [ ]:
existing_course_names = [course["Course"] for course in owned_courses]
courses_to_purchase = [course for course in courses_to_purchase if course["Course"] not in existing_course_names]

Temp!!!!!!!!!

In [ ]:
existing_course_names = [course["Course"] for course in owned_courses]
course_to_purchase = loadListItem("courses_sorted.txt")

used_courses = []
all_purchased_courses = []
for course in courses_list:
    if course["Course"] not in used_courses:
        used_courses.append(course["Course"])
        all_purchased_courses.append(course)
print(len(all_purchased_courses))

courses_to_purchase = [course for course in all_purchased_courses if course["Course"] not in existing_course_names]
print(len(courses_to_purchase))

courses_to_purchase = [course for course in courses_to_purchase if course["Course"] in course_to_purchase]
print(len(courses_to_purchase))


#courses_to_remove = ["Example Course Names", "That You Don't Want", "To Repurchase"]
courses_to_remove = ["Master Laravel 12 &amp; PHP: From Beginner to Advanced", 
                     "The Complete Hands-On Introduction to Apache Airflow 3"]
courses_to_purchase =  [course for course in courses_to_purchase if course["Course"] not in courses_to_remove]

print(len(courses_to_purchase))
saveJsonItem("courses_to_purchase.txt", courses_to_purchase)


# should be 125, but it's not so that's quite concerning... wonder if it's different named characters causing that... probably

In [ ]:
def addToCartTest(t_driver: uc.Chrome, t_courses_list: list, new_prices: bool = True) -> list:
    '''
    The function to add cheaper courses to the cart

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param t_courses_list: The list of courses with refunds available that are cheaper
    :type t_courses_list: list

    :param new_prices: If True will delete all cookies to get the new user sales 
    (which are usually cheaper than an existing user. You can still
    purchase courses from being logged out which keeps that coupon)
    :type new_prices: bool

    :return: The list of just courses with refunds available
    :rtype: list

    '''
    # Delete all cookies to get the new user sale price
    if(new_prices):
        t_driver.delete_all_cookies()
    # Go the course
    for ind, course in enumerate(t_courses_list):
        while True:
            t_driver.get(course["Course_Link"])
            time.sleep(course_page_pause/2 + random.randint(random_wait_min, random_wait_max))
            # Check if there's an add to cart button
            try:
                cart_button = t_driver.find_element(By.XPATH, ".//div[@class='buy-box--buy-box-item--wT5bJ buy-box--add-to-cart-button-wrapper--focmP']")
                cart_button = cart_button.find_element(By.XPATH, ".//button[@data-testid='add-to-cart-button']")
                print("Course {} Added to cart at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                course["New_Status"] = "Unowned"
                t_courses_list[ind] = course
                break
            except:
                # Check if we already own the course
                try:
                    course_button = t_driver.find_element(By.XPATH, ".//div[@class='buy-button buy-box--buy-box-item--wT5bJ buy-box--buy-button--m373K']")
                    course_button = course_button.find_element(By.XPATH, ".//button[@data-purpose='buy-this-course-button']")
                    course_button = course_button.find_element(By.XPATH, ".//span[@class='ud-btn-label']").text
                    print("Course {} Already Owned at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    course["New_Status"] = "Owned"
                    t_courses_list[ind] = course
                    break
                # If there's some kind of error we'll retry
                except:
                    print("Course {} Error at: {}".format(ind, datetime.datetime.strftime(datetime.datetime.now(), "%Y/%m/%d %H:%M:%S")))
                    course["New_Status"] = "Error"
                    t_courses_list[ind] = course
                    break
    return(t_courses_list)



In [ ]:
courses_to_purchase_test = addToCartTest(driver, courses_to_purchase, False)


In [ ]:
saveJsonItem("courses_to_purchase_test.txt", courses_to_purchase_test)

In [ ]:
courses_to_purchase_test_filter = [course for course in courses_to_purchase_test if course["New_Status"] == "Unowned"]


print(len(courses_to_purchase_test))
print(len(courses_to_purchase_test_filter))

In [ ]:
purchased_courses = addToCart(driver, courses_to_purchase_test_filter)

Remove any courses you see that shouldn't be repurchased

In [ ]:
courses_to_remove = ["Example Course Names", "That You Don't Want", "To Repurchase"]
courses_to_purchase =  [course for course in courses_to_purchase if course["Course"] not in courses_to_remove]
saveJsonItem("courses_to_purchase.txt", courses_to_purchase)

Add courses to the cart (You'll need to purchase them yourself still... I don't want to automate the actual purchase part... that seems dangerous)

In [ ]:
purchased_courses = addToCart(driver, courses_to_purchase)

Function to combine purchased courses to an instructor (and verify they weren't refunded in the past)

In [ ]:
# Basically join the two lists together on the course URL (probably make into a DataFrame first)
    # So I could've probably just got the course name from the purchase itself... maybe I'll
    # implement a function for that too because it'll be so simple.